In [2]:
import requests

In [3]:
#!python -m http.server

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
^C

Keyboard interrupt received, exiting.


In [17]:
consumer_key = 'i3vceldog8736e0nrc1b2ve886'

auth_url = f'https://secure.meetup.com/oauth2/authorize?client_id={consumer_key}&response_type=code&redirect_uri=http://127.0.0.1:8000/'

In [18]:
auth_response = requests.get(auth_url)

In [19]:
auth_response

<Response [403]>